In [2]:
try:
    import binutil
except ModuleNotFoundError:
    import bin.binutil

from dreamcoder.program import *
from dreamcoder.domains.relation import *
from dreamcoder.domains.relation.relation_primitives import *

get_baseline_primitives()
get_clevr_primitives()
get_clevr_primitives_unconfounded()

DEPRECATION NOTICE: this module (dreamcoder.ec) will be deleted soon, please update your code to import from dreamcoder.dreamcoder instead


In [4]:
# read txt file
mode = "test"
split = "color_count"  # material
same_support = True
seed = 0

if same_support:
    domain = "clevr_revised"
    data_folder = "curi"
else:
    domain = "clevr_single_support"
    data_folder = "curi_dc_test_tasks_single"

file = f"../consoleOutputs/{domain}/{split}/eval/{seed}/{mode}_programs.out"

with open(file, "r") as f:
    data = f.read()

if mode == "train":
    data = data.split("\n")
    data = [line for line in data if line != ""]
    task_programs = {}
    task_flag = False
    for line in data:
        if task_flag:
            program = line.split("\t")[1]
            task_programs[task_id] = program
            task_flag = False
        if "task" in line:
            task_flag = True
            task_id = line
else:
    data = data.split("\n")
    NUMBER_TEST_TASKS = len(data)
    print("Number of all tasks: ", len(data))
    task_programs = {}
    for task in data:
        if not "HIT" in task:
            continue
        task = task.split("w/")
        name = task[0].replace(" ", "")
        name = name.replace("HIT", "")
        program = task[1].split(";")[0][1:]
        task_programs[name] = program

Number of all tasks:  47


In [5]:
import pandas as pd
import json
import os
from tqdm import tqdm

path = f"../data/curi_dc_test_tasks/color_count/query/"
json_files = [f.path for f in os.scandir(path) if f.path.endswith(".json")]

task_results = []

# iterate over tasks
for task_file in tqdm(json_files):
    task_name = task_file.split("/")[-1][:-5]
    # task_number = task_name.split("_")[-1]
    # check if program exists for task
    if task_name in task_programs.keys():
        f = open(task_file)
        examples = json.load(f)

        parsed_examples = []

        true_positives = 0
        false_positives = 0
        true_negatives = 0
        false_negatives = 0

        for example in examples:
            input = example["input"]
            output = example["output"]

            # execute program for input
            program = task_programs[task_name]
            program = Program.parse(program)
            try:
                program_output = program.evaluate([])(input)
            except:
                continue

            # categorize prediction
            if output:
                if program_output:
                    true_positives += 1
                else:
                    false_negatives += 1
            else:
                if program_output:
                    false_positives += 1
                else:
                    true_negatives += 1

        # collect results
        results = {
            "task_name": task_name,
            "program": task_programs[task_name],
            "TP": true_positives,
            "FP": false_positives,
            "TN": true_negatives,
            "FN": false_negatives,
        }
        task_results.append(results)
        # print(results)
    else:
        continue
        results = {"task_name": task_name, "TP": 0, "FP": 0, "TN": 0, "FN": 0}
        task_results.append(results)

results_df = pd.DataFrame(task_results)
results_df

100%|██████████| 47/47 [01:56<00:00,  2.48s/it]


,task_name,program,TP,FP,TN,FN
0,task_465,(lambda (gt? (#(lambda (lambda (count (map (la...,948,3266,1754,307
1,task_8016,(lambda (eq? (#(lambda (lambda (count (map (la...,1255,5,5335,80
2,task_1676,(lambda (eq? (#(lambda (lambda (count (map (la...,1411,0,5920,69
3,task_2482,(lambda (#(lambda (lambda (and (#(lambda (exis...,567,554,2806,273
4,task_1377,(lambda (#(lambda (forall (lambda (gt? (index ...,1620,16,6464,0
5,task_3681,(lambda (#(lambda (exists (lambda (#(lambda (e...,354,1114,1986,421
6,task_3687,(lambda (gt? (#(lambda (lambda (count (map (la...,179,641,2859,696
7,task_7922,(lambda (gt? (#(lambda (lambda (count (map (la...,496,1020,4060,774
8,task_8521,(lambda (eq? (#(lambda (lambda (count (map (la...,1266,1,5139,19
9,task_8203,(lambda (eq? (#(lambda (lambda (count (map (la...,1116,0,4620,39


In [9]:
results_df["Accuracy"] = (results_df["TP"] + results_df["TN"]) / (
    results_df["TP"] + results_df["TN"] + results_df["FP"] + results_df["FN"]
)
results_df["CBA"] = (
    (results_df["TP"] / (results_df["TP"] + results_df["FN"]))
    + (results_df["TN"] / (results_df["TN"] + results_df["FP"]))
) / 2
results_df["Precision"] = results_df["TP"] / (results_df["TP"] + results_df["FP"])
# get number of tasks with accuracy not nan
len(results_df[~results_df["Accuracy"].isna()])

47

In [10]:
results_df.sort_values(by="CBA", ascending=True)

,task_name,program,TP,FP,TN,FN,Accuracy,CBA,Precision
36,task_669,(lambda (eq? (#(lambda (#(lambda (lambda (lamb...,332,1601,3599,968,0.604769,0.473750,0.171754
45,task_9280,(lambda (eq? (#(lambda (#(lambda (lambda (lamb...,225,912,1908,480,0.605106,0.497872,0.197889
13,task_1997,(lambda (eq? (#(lambda (lambda (#(lambda (lamb...,257,987,2713,668,0.642162,0.505541,0.206592
6,task_3687,(lambda (gt? (#(lambda (lambda (count (map (la...,179,641,2859,696,0.694400,0.510714,0.218293
32,task_3705,(lambda (eq? (#(lambda (lambda (lambda (+ $2 (...,197,681,2179,518,0.664615,0.518706,0.224374
5,task_3681,(lambda (#(lambda (exists (lambda (#(lambda (e...,354,1114,1986,421,0.603871,0.548710,0.241144
0,task_465,(lambda (gt? (#(lambda (lambda (count (map (la...,948,3266,1754,307,0.430598,0.552390,0.224964
7,task_7922,(lambda (gt? (#(lambda (lambda (count (map (la...,496,1020,4060,774,0.717480,0.594882,0.327177
20,task_8014,(lambda (#(lambda (not (#(lambda (lambda (fora...,1226,3481,1679,64,0.450388,0.637888,0.260463
17,task_463,(lambda (#(lambda (lambda (and (exists (lambda...,805,1623,3517,480,0.672685,0.655350,0.331549


In [11]:
print(split, " ", mode)
# get mean accuracy
print(results_df["Accuracy"].mean())
# get mean CBA
print(results_df["CBA"].mean())
# get precision
print(results_df["Precision"].mean())

color_count   test
0.879840540693081
0.8636580010276551
0.7615305704102243


In [12]:
print(split, " ", mode, " ", seed)
print("Class balanced accuracy (solved): ", results_df["CBA"].mean())

cba_50 = results_df["CBA"].mean() * (len(results_df) / NUMBER_TEST_TASKS) + 0.5 * (
    1 - (len(results_df) / NUMBER_TEST_TASKS)
)
# round to 2 decimal places
cba_50 = round(cba_50 * 100, 2)
print("Class balanced accuracy (all): ", cba_50)

color_count   test   0
Class balanced accuracy (solved):  0.8636580010276551
Class balanced accuracy (all):  86.37


In [8]:
# save results
results_df.to_csv(f"../experimentOutputs/{domain}/{split}_{mode}_{seed}.csv")
print("saved results to ", f"../experimentOutputs/{domain}/{split}_{mode}_{seed}.csv")

saved results to  ../experimentOutputs/clevr_revised/color_count_test_0.csv


In [15]:
import numpy as np

accs = [89.25, 82.8, 90.37]
print("mean accuracy over seeds: ", np.mean(accs))
print("std accuracy over seeds: ", np.std(accs))

print(round(np.mean(accs), 2), "$\pm$", round(np.std(accs), 2))

mean accuracy over seeds:  87.47333333333334
std accuracy over seeds:  3.336028910079903
87.47 $\pm$ 3.34
